## Precomputed_annotation_format
The purpose of this notebook is to show how to make a precomputed annotation layer from a list of 3d points.
Refer to this document: https://github.com/google/neuroglancer/blob/master/src/neuroglancer/datasource/precomputed/annotations.md for general structure (e.g. if you need another type of annotation than points)

In [7]:
import numpy as np
import os
import csv
import struct
import json
from cloudvolume import CloudVolume
import matplotlib.pyplot as plt
import neuroglancer
%matplotlib inline

In [3]:
# get the raw-space cells file and load it in
animal_id = 4
pth=os.path.join('/jukebox/wang/Jess/lightsheet_output',
        '201904_ymaze_cfos','processed',f'an{animal_id}','clearmap_cluster_output',
        'cells.npy')
converted_points = np.load(pth)

In [4]:
# Here is a test to show which method is best to use for making the annotation file. 
# Second method is MUCH faster -- use that one!

In [45]:
%%time
filename = '/home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0'

coordinates = converted_points # array like [[x1,y1,z1],[x2,y2,z2],...]
total_count = len(coordinates)
with open(filename,'wb') as outfile:
    buf = struct.pack('<Q',total_count)
    for (x,y,z) in coordinates:
        pt_buf = struct.pack('<3f',x,y,z)
        buf+=pt_buf
    id_buf = struct.pack('<%sQ' % len(coordinates), *range(len(coordinates)))
    buf+=id_buf
    outfile.write(buf)
print(f"wrote {filename}")

wrote /home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0
CPU times: user 3.98 s, sys: 16 ms, total: 3.99 s
Wall time: 3.97 s


In [47]:
%%time
filename = '/home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0'

coordinates = converted_points
total_count = len(coordinates)
with open(filename,'wb') as outfile:
    buf = struct.pack('<Q',total_count)
    pt_buf = b''.join(struct.pack('<3f',x,y,z) for (x,y,z) in coordinates)
    buf += pt_buf
    id_buf = struct.pack('<%sQ' % len(coordinates), *range(len(coordinates)))
    buf += id_buf
    outfile.write(buf)
print(f"wrote {filename}")

wrote /home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0
CPU times: user 584 ms, sys: 40 ms, total: 624 ms
Wall time: 622 ms
